In [1]:
# init the random seed and keras things
import numpy as np
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K

np.random.seed(1337)

Using TensorFlow backend.


In [2]:
# init the time function
import datetime

now = datetime.datetime.now

In [3]:
# hyperparameters
batch_size = 128
nb_classes = 5
nb_epoch = 5

In [4]:
# input image dimensions
img_rows, img_cols = 28, 28
# number of convolutional filters to use
nb_filters = 32
# size of pooling area for max pooling
pool_size = 2
# convolution kernel size
kernel_size = 3

if K.image_dim_ordering() == 'th':
    input_shape = (1, img_rows, img_cols)
else:
    input_shape = (img_rows, img_cols, 1)

In [5]:
# we use the model two times, so it is better to define a train
# function (modular programming! using abstraction)
def train_model(model, train, test, nb_classes):
    X_train = train[0].reshape((train[0].shape[0],) + input_shape)
    X_test = test[0].reshape((test[0].shape[0],) + input_shape)
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    X_train /= 255
    X_test /= 255
    print('X_train shape:', X_train.shape)
    print(X_train.shape[0], 'train samples')
    print(X_test.shape[0], 'test samples')

    # convert class vectors to binary class matrices
    Y_train = np_utils.to_categorical(train[1], nb_classes)
    Y_test = np_utils.to_categorical(test[1], nb_classes)

    model.compile(loss='categorical_crossentropy',
                  optimizer='adadelta',
                  metrics=['accuracy'])

    t = now()
    model.fit(X_train, Y_train,
              batch_size=batch_size, nb_epoch=nb_epoch,
              verbose=1,
              validation_data=(X_test, Y_test))
    print('Training time: %s' % (now() - t))
    score = model.evaluate(X_test, Y_test, verbose=0)
    print('Test score:', score[0])
    print('Test accuracy:', score[1])

In [6]:
# the data, shuffled and split between train and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [7]:
# create two datasets one with digits below 5 and one with 5 and above
X_train_lt5 = X_train[y_train < 5]
y_train_lt5 = y_train[y_train < 5]
X_test_lt5 = X_test[y_test < 5]
y_test_lt5 = y_test[y_test < 5]

X_train_gte5 = X_train[y_train >= 5]
y_train_gte5 = y_train[y_train >= 5] - 5  # make classes start at 0 for
X_test_gte5 = X_test[y_test >= 5]         # np_utils.to_categorical
y_test_gte5 = y_test[y_test >= 5] - 5

In [8]:
# define two groups of layers: feature (convolutions) and classification (dense)
feature_layers = [
    Convolution2D(nb_filters, kernel_size, kernel_size,
                  border_mode='valid',
                  input_shape=input_shape),
    Activation('relu'),
    Convolution2D(nb_filters, kernel_size, kernel_size),
    Activation('relu'),
    MaxPooling2D(pool_size=(pool_size, pool_size)),
    Dropout(0.25),
    Flatten()
]
classification_layers = [
    Dense(128),
    Activation('relu'),
    Dropout(0.5),
    Dense(nb_classes),
    Activation('softmax')
]

In [9]:
# create complete model, use list here instead of series of .add()
model = Sequential(feature_layers + classification_layers)

In [10]:
# train model for 5-digit classification [0..4]
train_model(model,
            (X_train_lt5, y_train_lt5),
            (X_test_lt5, y_test_lt5), nb_classes)

X_train shape: (30596, 28, 28, 1)
30596 train samples
5139 test samples
Train on 30596 samples, validate on 5139 samples
Epoch 1/5
30596/30596 [==============================] - 5s - loss: 0.2149 - acc: 0.9348 - val_loss: 0.0398 - val_acc: 0.9872
Epoch 2/5
30596/30596 [==============================] - 3s - loss: 0.0627 - acc: 0.9812 - val_loss: 0.0233 - val_acc: 0.9922
Epoch 3/5
30596/30596 [==============================] - 3s - loss: 0.0447 - acc: 0.9870 - val_loss: 0.0192 - val_acc: 0.9926
Epoch 4/5
30596/30596 [==============================] - 3s - loss: 0.0361 - acc: 0.9885 - val_loss: 0.0148 - val_acc: 0.9946
Epoch 5/5
30596/30596 [==============================] - 3s - loss: 0.0300 - acc: 0.9912 - val_loss: 0.0152 - val_acc: 0.9947
Training time: 0:00:18.610983
Test score: 0.0151647937127
Test accuracy: 0.994746059545


In [11]:
# freeze feature layers and rebuild model
for l in feature_layers:
    l.trainable = False

In [12]:
# transfer: train dense layers for new classification task [5..9]
train_model(model,
            (X_train_gte5, y_train_gte5),
            (X_test_gte5, y_test_gte5), nb_classes)

X_train shape: (29404, 28, 28, 1)
29404 train samples
4861 test samples
Train on 29404 samples, validate on 4861 samples
Epoch 1/5
29404/29404 [==============================] - 2s - loss: 0.3631 - acc: 0.8932 - val_loss: 0.0922 - val_acc: 0.9685
Epoch 2/5
29404/29404 [==============================] - 1s - loss: 0.1226 - acc: 0.9627 - val_loss: 0.0599 - val_acc: 0.9803
Epoch 3/5
29404/29404 [==============================] - 1s - loss: 0.0916 - acc: 0.9737 - val_loss: 0.0476 - val_acc: 0.9846
Epoch 4/5
29404/29404 [==============================] - 1s - loss: 0.0744 - acc: 0.9775 - val_loss: 0.0397 - val_acc: 0.9864
Epoch 5/5
29404/29404 [==============================] - 1s - loss: 0.0649 - acc: 0.9805 - val_loss: 0.0359 - val_acc: 0.9879
Training time: 0:00:10.612115
Test score: 0.0358572804061
Test accuracy: 0.987862579606


+ Though we dont make the network learn how to identify numbers 5->9, the features extracted from the output of convolution layers (0->4) are enough to classify the numbers 5->9! Amazing!